## Nome: Felipe Silva Loschi
## Matricula: 601
-----


# **ATV — Testes Categorizados**

Esta atividade pratica os **testes para dados categorizados**: Qui‑quadrado clássico, Qui‑quadrado com correção de Yates, Teste Exato de Fisher e Teste de McNemar.

**Objetivos**
- Montar tabelas de contingência e formular hipóteses (H0/H1).
- Calcular valores **esperados** e a estatística **χ²** (ou **p‑valor**), decidir e concluir.
- Usar `scipy.stats`/`statsmodels` para conferir os cálculos.
- Interpretar resultados nos níveis usuais de significância (1%, 5%, 10%).




## Instruções gerais

- Execute as células na ordem em que aparecem.
- Sempre **declare as hipóteses** antes de calcular (H0 = igualdade; H1 = diferença).
- Para tabelas 2×2, você pode usar:
  - `stats.chi2_contingency(obs, correction=False)` → χ² clássico;
  - `stats.chi2_contingency(obs, correction=True)` → com **Yates**;
  - `stats.fisher_exact(obs, alternative="two-sided")` → **Fisher** (n pequeno ou valores esperados < 5);
  - `mcnemar(tabela_pareada, exact=True/False, correction=True/False)` → **McNemar** (amostras pareadas).

In [2]:
# Imports úteis para toda a atividade
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.contingency_tables import mcnemar

pd.set_option("display.precision", 4)



## Funções auxiliares


In [3]:
def esperados_from_obs(obs):
    obs = np.asarray(obs, dtype=float)
    row_sum = obs.sum(axis=1, keepdims=True)
    col_sum = obs.sum(axis=0, keepdims=True)
    total = obs.sum()
    return row_sum @ col_sum / total

def decide_from_p(p, alphas=(0.10, 0.05, 0.01)):
    return {a: ("Rejeita H0" if p < a else "Não rejeita H0") for a in alphas}




## Exercício 1 — Qui‑quadrado **clássico** (amostras independentes, **n > 40**)

**Contexto**: Ensaio clínico com pacientes com dor abdominal. Grupo **Tratamento** recebeu brometo de pinavério; grupo **Controle** recebeu placebo.  
Tabela de **valores observados** (2×2), **n = 154**:

|                 | **Dor Sim** | **Dor Não** | **Total** |
|-----------------|:-----------:|:-----------:|:---------:|
| **Tratamento**  |      6      |     57      |    63     |
| **Controle**    |     30      |     61      |    91     |
| **Total**       |     36      |    118      |   154     |

1. Formule H0 e H1.
2. Calcule a **tabela de esperados** e a estatística **χ² calculado** (sem correção).
3. Obtenha **gl**, **χ² tabelado** para α∈{10%,5%,1%} (use SciPy) e o **p‑valor**.
4. Decida e conclua sobre a **eficácia do tratamento**.


**H0**: Hipótese de nulidade, ou seja, não há diferença entre os grupos, o que indicaria que o tratamento não é eficaz.

**H1**: Hipótese alternativa, que defenderia que há diferença entre os grupos, logo, o tratamento seria eficaz.

In [4]:

# Tabela observada (Exemplo 1)
obs1 = np.array([[6, 57],
                 [30, 61]])
obs1_df = pd.DataFrame(obs1, index=["Tratamento","Controle"], columns=["Dor_Sim","Dor_Nao"])


In [5]:

# χ² clássico (sem correção de Yates) e esperados
chi2, p, dof, expected = stats.chi2_contingency(obs1, correction=False)
expected_df = pd.DataFrame(expected, index=obs1_df.index, columns=obs1_df.columns)



In [6]:

# χ² tabelado (crítico) para níveis usuais
alphas = [0.10, 0.05, 0.01]
chi2_crit = {a: stats.chi2.ppf(1 - a, dof) for a in alphas}


In [7]:
print("Teste Qui-Quadrado Clássico")
print("\nTabela Esperada:")
print(expected_df)
print(f"\nEstatística Qui-Quadrado Calculada: {chi2:.4f}")
print(f"Graus de Liberdade: {dof}")
print("\nQui-Quadrado C rítico para diferentes níveis de significância:")
for a in alphas:
    print(f"  α = {a}: χ² crítico = {chi2_crit[a]:.4f}")
print(f"\nValor-p: {p:.4f}")
decisions = decide_from_p(p, alphas)
print("\nDecisão com base no valor-p:")
for a in alphas:
    print(f"  α = {a}: {decisions[a]}")

Teste Qui-Quadrado Clássico

Tabela Esperada:
            Dor_Sim  Dor_Nao
Tratamento  14.7273  48.2727
Controle    21.2727  69.7273

Estatística Qui-Quadrado Calculada: 11.4223
Graus de Liberdade: 1

Qui-Quadrado C rítico para diferentes níveis de significância:
  α = 0.1: χ² crítico = 2.7055
  α = 0.05: χ² crítico = 3.8415
  α = 0.01: χ² crítico = 6.6349

Valor-p: 0.0007

Decisão com base no valor-p:
  α = 0.1: Rejeita H0
  α = 0.05: Rejeita H0
  α = 0.01: Rejeita H0


**Conclusão:** Com os dados obtidos, observa-se que rejeitamos H0, ou seja, rejeita-se a hipótese de igualdade. Com isso, concluí-se que há diferença entre o grupo que recebeu o tratamento e o grupo que recebeu placebo, provando então que o tratamento é eficaz.



## Exercício 2 — Qui‑quadrado **com correção de Yates** (amostras independentes, **20 < n < 40**)

Considere a tabela 2×2 abaixo com **n = 32** (dados fictícios para prática).

| Grupo | Sucesso | Fracasso | Total |
|-------|:-------:|:--------:|:-----:|
| A     |   14    |    2     |  16   |
| B     |    6    |   10     |  16   |
| **Total** | 20 | 12 | 32 |

1. Formule H0 e H1.
2. Calcule os **esperados** e **χ²** com **correção de Yates**.
3. Informe **gl**, **p‑valor** e **decisão** para α∈{10%,5%}.


**H0**: Hipótese de nulidade, ou seja, não há diferença entre os grupos A e B.

**H1**: Hipótese alternativa, que nesse caso  indicaria que há diferença entre o sucesso e fracasso dos grupos A e B.

In [8]:

obs2 = np.array([[14, 2],
                 [ 6,10]])
obs2_df = pd.DataFrame(obs2, index=["A","B"], columns=["Sucesso","Fracasso"])

# χ² com correção de Yates
chi2_y, p_y, dof_y, exp2 = stats.chi2_contingency(obs2, correction=True)



In [9]:

alphas = [0.10, 0.05]
crit2 = {a: stats.chi2.ppf(1 - a, dof_y) for a in alphas}


In [10]:
print("\n\nTeste Qui-Quadrado com Correção de Yates")
print("\nTabela Esperada:")
expected2_df = pd.DataFrame(exp2, index=obs2_df.index, columns=obs2_df.columns)
print(expected2_df)
print(f"\nEstatística Qui-Quadrado com Correção de Yates Calculada: {chi2_y:.4f}")
print(f"Graus de Liberdade: {dof_y}")
print("\nQui-Quadrado Crítico para diferentes níveis de significância:")
for a in alphas:
    print(f"  α = {a}: χ² crítico = {crit2[a]:.4f}")
print(f"\nValor-p: {p_y:.4f}")
decisions_y = decide_from_p(p_y, alphas)
print("\nDecisão com base no valor-p:")
for a in alphas:
    print(f"  α = {a}: {decisions_y[a]}")



Teste Qui-Quadrado com Correção de Yates

Tabela Esperada:
   Sucesso  Fracasso
A     10.0       6.0
B     10.0       6.0

Estatística Qui-Quadrado com Correção de Yates Calculada: 6.5333
Graus de Liberdade: 1

Qui-Quadrado Crítico para diferentes níveis de significância:
  α = 0.1: χ² crítico = 2.7055
  α = 0.05: χ² crítico = 3.8415

Valor-p: 0.0106

Decisão com base no valor-p:
  α = 0.1: Rejeita H0
  α = 0.05: Rejeita H0




**Conclusão:** Conclui-se, portanto, que há uma diferença significativa entre os grupos A e B, visto que o valor de p indica para uma diferença significante de diferença entre os grupos e de alta certeza de rejeição de H0, que seria a hipótese de nulidade de diferença. Logo, conclui-se que há uma diferença grande entre os grupos.



## Exercício 3 — **Teste Exato de Fisher** (amostras independentes, **n < 20** ou esperados < 5)

**Sobrevida de ratos** após 1,5 ano:

|           | **Vivos** | **Mortos** | **Total** |
|-----------|:---------:|:----------:|:---------:|
| **Normal**      |     5     |     2      |    7     |
| **Experimental**|     1     |     8      |    9     |
| **Total**       |     6     |    10      |    16    |

1. Formule H0 e H1 (duas caudas).
2. Aplique **Fisher** (two‑sided) → **p‑valor**.
3. Decida em α=5% e conclua.


**H0**: Hipótese de nulidade, que nesse caso indicaria que não há diferença na sobrevida entre os ratos normais e os ratos que foram experimentados.

**H1**: Hipótese alternativa, que nesse caso indicaria que há uma diferença na sobrevida dos ratos que foram experimentados em relação aos ratos normais.

In [11]:

obs3 = np.array([[5,2],
                 [1,8]])
oddsratio, p_fisher = stats.fisher_exact(obs3, alternative="two-sided")


In [17]:
print("\n\nTeste Exato de Fisher")
print(f"\nOdds Ratio: {oddsratio:.4f}")
print(f"Valor-p: {p_fisher:.4f}")
decisions_fisher = decide_from_p(p_fisher, alphas)
print("\nDecisão com base no valor-p:")
for a in alphas:
    print(f"  α = {a}: {decisions_fisher[a]}")



Teste Exato de Fisher

Odds Ratio: 20.0000
Valor-p: 0.0350

Decisão com base no valor-p:
  α = 0.1: Rejeita H0
  α = 0.05: Rejeita H0



**Conclusão:** Tomando α=5% e decidindo com base no valor-p, decide-se pela rejeição de H0, o que indica que há uma diferença na sobrevida dos ratos que foram experimentados em relação aos ratos normais.


## Exercício 4 — **Teste de McNemar** (amostras pareadas: antes × depois)

**Teste de memória (n = 40)** — número de acertos **completos** (15 palavras) antes e depois do fármaco:

Tabela **pareada** (linhas = Antes; colunas = Depois):

|             | **Correto** | **Incorreto** | **Total** |
|-------------|:-----------:|:-------------:|:---------:|
| **Correto** |      8      |       1       |     9     |
| **Incorreto** |    14     |      17       |    31     |
| **Total**   |     22      |      18       |    40     |

1. Formule H0 e H1 (duas caudas).
2. Calcule McNemar com e sem correção de Yates; reporte **estatística** e **p‑valor**.
3. Decida (α=1% e α=5%) e conclua.


**H0**: Hipótese de nulidade, que nesse caso indicaria que não diferença no teste antes e depois do fármaco.

**H1**: Hipótese alternativa, que nesse caso indicaria que há diferença no teste antes e depois do fármaco.

In [13]:

tab = np.array([[8,1],[14,17]])
res_corr = mcnemar(tab, exact=False, correction=True)
res_semc = mcnemar(tab, exact=False, correction=False)


In [19]:
print("\n\nTeste de McNemar")
alphas_mc = [0.05, 0.01]  # α=5% e α=1% conforme solicitado

print("resultado com correção de Yates:")
print(f"\nEstatística de McNemar com Correção de Yates: {res_corr.statistic:.4f}")
print(f"Valor-p: {res_corr.pvalue:.4f}")
for a in alphas_mc:
    decisao = "Rejeita H0" if res_corr.pvalue < a else "Não rejeita H0"
    print(f"  α = {a}: {decisao}")

print("\nresultado sem correção de Yates:")
print(f"\nEstatística de McNemar sem Correção de Yates: {res_semc.statistic:.4f}")
print(f"Valor-p: {res_semc.pvalue:.4f}")
for a in alphas_mc:
    decisao = "Rejeita H0" if res_semc.pvalue < a else "Não rejeita H0"
    print(f"  α = {a}: {decisao}")



Teste de McNemar
resultado com correção de Yates:

Estatística de McNemar com Correção de Yates: 9.6000
Valor-p: 0.0019
  α = 0.05: Rejeita H0
  α = 0.01: Rejeita H0

resultado sem correção de Yates:

Estatística de McNemar sem Correção de Yates: 11.2667
Valor-p: 0.0008
  α = 0.05: Rejeita H0
  α = 0.01: Rejeita H0



**Conclusão:** Tanto com correção de Yates quanto sem correção, e em ambos os níveis de significância (α=5% e α=1%), rejeita-se H0. Nesse caso, isso indica que há diferenças significativas no teste antes e depois do fármaco.